# Preparation et chargement du corpus

In [2]:
from pathlib import Path
import re


data_dir = Path("../../data/txt")

all_txt = list(data_dir.glob("*.txt"))

YEAR = "1950"
pat = re.compile(rf"{YEAR}")

files_year = [p for p in all_txt if pat.search(p.name)]
print(f"Fichiers trouvés pour {YEAR}: {len(files_year)}")

corpus_year = ""
for path in files_year:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            with open(path, "r", encoding=enc, errors="ignore") as f:
                corpus_year += f.read() + "\n"
            break
        except Exception:
            continue

print(f"Taille du corpus {YEAR}: {len(corpus_year):,} caractères")
print("\nExtrait:\n", corpus_year[:500])

Fichiers trouvés pour 1950: 4
Taille du corpus 1950: 133,895 caractères

Extrait:
 L'AVENIR DU LUXEMBOURG Samedi 15 avri j 350, 
MORHET 
Soirée dramatique 
1 Le cercle dramatique Sainte-Cécile 
de Morhet reprendra, ce dimanche 16 
avril ^Quasimodo), sa brillante soirée 
qui a remporté un succès si remarqua-
| bie le 10 mars dernier. 
i Rappelons ie programme : 
; 1) ouverture : « Brabançonne »,par 
• la Fantare ; 2) « La .bohème », chœur 
à 2 voix exécuté par JV^.es Renée Cara, 
j Josée Goffin, Anyse Hubermont et Hé-
f lène Bellanger ; a) La comédie en deux 
actes de Marcell* 


# Nuages de mots

## Imports et stopwords

In [3]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

[nltk_data] Downloading package stopwords to C:\Users\Ing Armel
[nltk_data]     Fopa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "journal", "édition", 
       "bruxelles", "belgique", "numéro", "année", "article", "soir", "camille"]
sw = set(sw)

print(f"Nombre total de stopwords: {len(sw)}")

Nombre total de stopwords: 209


Nettoyage léger du texte (Préparation WordCloud)

In [5]:
cleaned = re.sub(r"[^a-zA-ZÀ-ÖØ-öø-ÿ' -]", "", corpus_year).lower()
cleaned = re.sub(r"\s+", "", cleaned).strip()

len(cleaned)

95481

Génération du nuage de mots

In [6]:
wc = WordCloud(width=1200, height=600,
               background_color="write",
               stopwords=sw,
               collocations=False).generate(cleaned)

## Créer un fichier contenant le texte de tous les journaux d'une année donnée

In [ ]:
# Choisir une année
year = 1945

In [ ]:
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
# Compter le nombre d'éléments (=fichiers) dans la liste
len(content_list)

In [ ]:
# Imprimer les 200 premiers caractères du contenu du premier fichier
content_list[0][0:200]

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

## Nettoyer le fichier à l'aide d'une fonction de nettoyage

### Créer la fonction de nettoyage (à adapter)

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

### Appliquer la fonction sur le fichier complet de l'année

In [ ]:
clean_text(year, folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

## Nuage de mots

### Afficher les termes les plus fréquents


In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(10))

### Créer, stocker et afficher le nuage de mots

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))